In [2]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=z1nYqGtU6IeGZbsoBQFYJjHJ3T3qYAmZd3kGn7BC"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2024-12-22T13:34:34+00:00","rankings":[{"type_id":2,"name":"ATP","year":2024,"week":51,"gender":"men","competitor_rankings":[{"rank":1,"movement":0,"points":7510,"competitions_played":23,"competitor":{"id":"sr:competitor:49363","name":"Pavic, Mate","country":"Croatia","country_code":"HRV","abbreviation":"PAV"}},{"rank":1,"movement":0,"points":7510,"competitions_played":23,"competitor":{"id":"sr:competitor:51836","name":"Arevalo-Gonzalez, Marcelo","country":"El Salvador","country_code":"SLV","abbreviation":"ARE"}},{"rank":3,"movement":0,"points":6655,"competitions_played":22,"competitor":{"id":"sr:competitor:87690","name":"Thompson, Jordan","country":"Australia","country_code":"AUS","abbreviation":"THO"}},{"rank":4,"movement":0,"points":6500,"competitions_played":18,"competitor":{"id":"sr:competitor:15568","name":"Granollers, Marcel","country":"Spain","country_code":"ESP","abbreviation":"GRA"}},{"rank":4,"movement":0,"points":6500,"competitions_played":18,"competitor":{

In [3]:
data = response.json()

In [ ]:
if response.status_code == 200:
    try:
        data = response.json()

        cr = []
        auto_increment_id = 1

        # Iterate over rankings to extract the required data
        for ranking in data.get("rankings", []):
            for competitor_ranking in ranking.get("competitor_rankings", []):
                competitor = competitor_ranking.get("competitor", {})
                cr.append({
                    "rank_id": auto_increment_id,
                    "competitor_rank": competitor_ranking.get("rank"),
                    "movement": competitor_ranking.get("movement"),
                    "points": competitor_ranking.get("points"),
                    "competitions_played": competitor_ranking.get("competitions_played"),
                    "competitor_id": competitor.get("id"),
                    "name": competitor.get("name"),
                    "country": competitor.get("country"),
                    "week": ranking.get("week"),  # Adjusted to get 'week' from the parent ranking object
                    "country_code": competitor.get("country_code"),
                    "abbreviation": competitor.get("abbreviation"),
                })
                auto_increment_id += 1

        # Output the extracted data
        for item in cr:
            print(item)
    except Exception as e:
        print(f"Error processing data: {e}")
else:
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

In [ ]:
import pandas as pd
df=pd.DataFrame(cr)
df

In [6]:
!pip install mysql-connector-python

In [15]:
import mysql.connector

# Establishing the connection to the local MySQL server
connection = mysql.connector.connect(
    host="localhost",         # MySQL server host (localhost for local machine)
    user="root",     # Your MySQL username
    password="Umaneethi@123", # Your MySQL password
    database="sportsradar"  # Database name you want to connect to
)

In [16]:
cursor = connection.cursor()

In [17]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS competitor_ranking_table (
            rank_id INT ,
            competitor_rank INT,
            movement INT,
            points INT,
            competitions_played INT,
            competitor_id VARCHAR(255)
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS competitors_table (
        competitor_id VARCHAR(255),
        name VARCHAR(255),
        country VARCHAR(255),
        week VARCHAR(50),       
        country_code VARCHAR(10),
        abbreviation VARCHAR(10)
    )
""")


In [10]:
## Insert data into competitor_ranking_table
ranking_data = [(item['rank_id'],
                 item['competitor_rank'],
                 item['movement'],
                 item['points'],
                 item['competitions_played'],
                 item['competitor_id']) for item in cr]

#This list comprehension loops over the cr list (which contains ranking data) and extracts relevant fields. 
#Each item represents a dictionary with keys like 'rank_id', 'competitor_rank', etc.
#The result is a list of tuples ready for batch insertion.

cursor.executemany("""
    INSERT INTO competitor_ranking_table (rank_id, competitor_rank, movement, points, competitions_played, competitor_id)
    VALUES (%s, %s, %s, %s, %s, %s)
""", ranking_data) #executemany inserts multiple rows into the competitor_ranking_table at once.

# Insert data into competitors_table
competitor_data = [(item['competitor_id'],
                    item['name'],
                    item['country'],
                    item['week'],
                    item['country_code'],
                    item['abbreviation']) for item in cr]


cursor.executemany("""
    INSERT INTO competitors_table (competitor_id, name, country, week, country_code, abbreviation)
    VALUES (%s, %s, %s, %s, %s, %s)
""", competitor_data)

# Commit changes to the database
connection.commit()

print("Data inserted successfully!")

# Close the connection
cursor.close()
connection.close()


Data inserted successfully!


In [11]:
!pip install tabulate

In [12]:
from tabulate import tabulate

In [ ]:
#Get all competitors with their rank and points.
cursor.execute(
    '''
    SELECT competitor_id, competitor_rank, points
    FROM competitor_ranking_table
    JOIN competitors_table USING (competitor_id)
    '''
)
results = cursor.fetchall()
headers = ["Competitor_id", "Competitor_rank", "Points"]
table = tabulate(results, headers, tablefmt="pipe")
print(table)


In [ ]:
#Find competitors ranked in the top 5
cursor.execute(
    '''
    SELECT competitor_id, competitor_rank
    FROM competitor_ranking_table
    ORDER BY competitor_rank
    Limit 5
    '''
)
results = cursor.fetchall()
headers = ["Competitor_id", "Competitor_rank"]
table = tabulate(results, headers, tablefmt="pipe")
print(table)

In [ ]:
#List competitors with no rank movement (stable rank)
cursor.execute(
    '''select competitor_id,competitor_rank ,movement  from competitor_ranking_table where movement = 0

    '''             
)
results = cursor.fetchall()
headers = ["Competitor_id","Competitor_rank " ,"Movement"]
table = tabulate(results, headers, tablefmt="pipe")
print(table)

In [38]:
#Get the total points of competitors from a specific country (e.g., Croatia)
cursor.execute(
    '''select sum(points) as Total_points_of_competitors, country from competitor_ranking_table r
join competitors_table c on r.competitor_id = c.competitor_id
group by country
having country = "Croatia"
    '''             
)
results = cursor.fetchall()
headers = ["Total_points_of_competitors", "country"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+-------------------------------+-----------+
|   Total_points_of_competitors | country   |
+===============================+===========+
|                         17061 | Croatia   |
+-------------------------------+-----------+


In [ ]:
#Count the number of competitors per country
cursor.execute(
    '''
    SELECT c.country, COUNT(*) AS total_competitors
    FROM competitor_ranking_table r
    JOIN competitors_table c ON r.competitor_id = c.competitor_id
    GROUP BY c.country
    '''
)
results = cursor.fetchall()
headers = ["total_competitors", "country"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

In [43]:
##6.Find competitors with the highest points in the current week
cursor.execute(
    '''
    SELECT week AS current_week, MAX(points) AS highest_points
    FROM competitors_table a
    JOIN competitor_ranking_table b
    ON a.competitor_id = b.competitor_id
    GROUP BY week
    '''
)
results = cursor.fetchall()
headers = ["highest_points", "current_week"]
table = tabulate(results, headers, tablefmt="grid")
print(table)

+------------------+----------------+
|   highest_points |   current_week |
+==================+================+
|               51 |           9530 |
+------------------+----------------+
